# Natural Language Processing for Signal Generation on News Data

### Neural Networks Model

In [ ]:
import datetime
import pydot, graphviz
from keras import layers
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import LSTM, concatenate, Bidirectional
from keras.layers import PReLU, ELU, LeakyReLU, GRU, SimpleRNN
from keras.layers import Input, Dense, Dropout, Activation, Embedding, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.utils.vis_utils import plot_model

### Network for processing sequential information

In [ ]:
## Subchannel network for encoding sequential information
def subnetwork_channel(input_layer : layers, RNN_architecture : str, units : int, dropout_rate : float) -> layers:
    """
    This function creates a sub network for encoding sequences.
    
    Inputs:
    input_layer - The input keras layer into the subnetwork
    RNN_architecture - Name of the RNN type to use
    units - Number of units in the RNN
    dropout_rate - dropout rate
    
    Outputs:
    batch - Batch Normalized output layer
    
    """
    assert RNN_architecture in ["LSTM", "GRU", "RNN"]
    
    dropout1 = Dropout(rate = dropout_rate)(input_layer)
    
    if RNN_architecture == "LSTM":
        rnn_layer = Bidirectional(LSTM(units = units, return_sequences = False))(dropout1)
    elif RNN_architecture == "GRU":
        rnn_layer = Bidirectional(GRU(units = units, return_sequences = False))(dropout1)
    elif RNN_architecture == "RNN":
        rnn_layer = Bidirectional(SimpleRNN(units = units, return_sequences = False))(dropout1)
    
    dropout2 = Dropout(rate = dropout_rate)(rnn_layer)
    batch = BatchNormalization()(dropout2)
    return batch

### Network for Generating Signal

In [ ]:
## Output layer network
def output_channel(input_layer : layers ,activation : str, units : int, dropout_rate : float) -> layers:
    """
    This function creates a sub network for outputing classification probabilities.
    
    Inputs:
    input_layer - The input keras layer into the subnetwork
    activation  - Name of the activation type to use
    units - Number of units in the Dense network
    dropout_rate - dropout rate
    
    Outputs:
    output - Softmax output layer
    
    """
    assert activation in ["ReLU","PReLU", "ELU", "LeakyReLU"]
    
    dense = Dense(units)(input_layer)
    
    if activation == "PReLU":
        act = PReLU()(dense)
    elif activation == "ELU":
        act = ELU()(dense)
    elif activation == "LeakyReLU":
        act = LeakyReLU()(dense)
    elif activation == "ReLU":
        act = Dense(units, activation='relu')(input_layer)
    
        
    dropout = Dropout(rate = dropout_rate)(act)
    batch = BatchNormalization()(dropout)
    output = Dense(3,activation='softmax', name = "Output")(batch)
    
    return output

### Put All Parts Together

In [ ]:
## Define full model.
def define_model(RNN_architecture : str = "LSTM", rnn_units : int = 256, dense_units : int = 128,dense_activation : str = "PReLU" ,dropout_rate : float = 0.4) -> Model:
    """
    This function defines and compiles a Multichannel RNN for Sentiment Classification.
    
    Inputs:
    RNN_architecture - Name of the RNN type to use
    rnn_units - Number of units in the RNN
    dense_units - Number of units in the Dense network
    dense_activation  - Name of the activation type to use
    dropout_rate - dropout rate
    
    Outputs:
    model - A Keras model
    
    """
    # Input Layer
    shape = (MAX_SEQUENCE_LENGTH,)
    input1 = Input(shape = shape, name = "Main_input")
    
    # Channel 1 - GLoVe
    embedding1 = Embedding(len(word_index) + 1,
              EMBEDDING_DIM,
              weights=[glove_embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH,
              trainable=False,
              input_shape=X_train.shape[1:], name = "GLoVe_Embedding")(input1)

    net1 = subnetwork_channel(embedding1, RNN_architecture = RNN_architecture, units = rnn_units, dropout_rate = dropout_rate)
    
    # Channel 2 - Fast Text
    embedding2 = Embedding(len(word_index) + 1,
              EMBEDDING_DIM,
              weights=[fasttext_embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH,
              trainable=False,
              input_shape=shape, name = "FastText_Embedding")(input1)

    net2 = subnetwork_channel(embedding2, RNN_architecture = RNN_architecture, units = rnn_units, dropout_rate = dropout_rate)
    
    # Merge
    merged = concatenate([net1,net2], name ="Merge")
    # Output channel
    output = output_channel(merged, activation = dense_activation, units = dense_units, dropout_rate = dropout_rate)
    
    # Compile 
    model = Model(inputs = input1, outputs = output)
    model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.002), metrics = ['categorical_accuracy'])
    
    return model

### Initialize the Model

In [ ]:

model = define_model(RNN_architecture = "LSTM",
                     rnn_units= 256,
                     dense_units = 128,
                     dense_activation = "ReLU",
                     dropout_rate = 0.4)
model.summary()


In [ ]:
pic_name = 'imgs/multichannel-bidirectionalLSTM.png'
plot_model(model,show_shapes=True,to_file=pic_name)

<img src="../imgs/multichannel-bidirectionalLSTM.png">

### Training

In [ ]:
## Train
tensorboard = TensorBoard(log_dir='tasks/tensorboard/logs/')
model.fit(X_train, y_train, epochs = 10, batch_size = 1024, callbacks = [tensorboard])

### Click [here](/tensorboard/) to start TensorBoard.

### testing

In [ ]:
## Find the testing accuracy
val_loss, val_catergorical_accuracy = model.evaluate(X_test,y_test)
print("Validation Accuracy: {:.1f}".format(val_catergorical_accuracy * 100))

Our model was able to achieve ~79% accuracy. According to research on sentiment analysis and classification, human raters may only agree with each other about 80% of the time. Due to the nature of sentiment analysis, the outcome a reader arrives at can be very subjective depending on how the reader interprets the words, tone or phrasing of the text. Thus, a model that predicts with 100% accuracy may still disagree with a human 20% of the time. 

### Exercise: Re-tune Neural Network Parameters
Try experimenting with different parameters in the neural network.
In the function 'define_model'
    - 'RNN_architecture' can be one of: "RNN", "GRU", "LSTM".
    - 'rnn_units' are the number of units in the RNN
    - 'dense_units' are the number of units in the dense network
    - 'dense_activation' can be one of: "PReLU", "LeakyReLU", "ELU", "ReLU"
    - 'dropout_rate' rate of dropout throughout the network